In [1]:
import os
import sys
import pickle
import numpy as np
from math import pi
from packages import data_container
from packages.data_container import Data
from packages.helper import traj_speed, beta
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

In [2]:
'''Import data from csv to Data object'''
header=('subj_x', 'subj_y', 'subj_z', 'subj_yaw', 'subj_pitch', 'subj_row')
info={'p_goal':[], 'p_obst':[], 'obst_angle':[], 'obst_speed':[], 'subj_id':[], 'trial_id':[], 'stimuli_onset':[], 
      'stimuli_out':[], 'w_goal': 0.4, 'w_obst': 0.2, 'ps_trial':[], 'ps_subj':[], 'time_stamp': []}
data = Data(Hz=90, header=header, info=info)
n_subj = 4
for subj_id in range(n_subj):
    # Import output
    file_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Subj' + str(subj_id).zfill(2) + '.csv'))
    with open(file_path, 'r') as f:
        trials = [line.split(',') for line in f.read().split('\n')]
        head = 0
        while head + 1 < len(trials):
            for i in range(head+1, len(trials)):
                if trials[i][0] == '':
                    tail = i - 1
                    break
            trial_id = int(trials[head+1][1])
            angle = float(trials[head+1][3])
            speed = float(trials[head+1][5])
            subj = np.array([[float(val) for val in line[1:7]] for line in trials[head+3:tail+1]])
            goal = np.array([[float(line[-2]), float(line[-1])] for line in trials[head+3:tail+1]])
            obst = np.array([[float(line[-4]), float(line[-3])] for line in trials[head+3:tail+1]])
            time_stamp = [float(line[0]) for line in trials[head+3:tail+1]]
            for i in range(len(obst) - 1):
                if obst[i][0] != obst[i + 1][0] or obst[i][1] != obst[i + 1][1]:
                    onset = i
                    break
            for i in range(onset, len(subj) - 1):
                p0 = subj[i, [0,2]]
                p1 = obst[i]
                v0 = (subj[i+1,[0,2]] - subj[i,[0,2]]) * data.Hz
                if abs(beta(p0, p1, v0)) > pi / 2:
                    stimuli_out = i
                    break
            data.add_traj(subj)
            ps_trial = np.mean(traj_speed(subj[:,[0,2]], data.Hz)[-2*data.Hz:-data.Hz])
            data.add_info({'p_goal':goal, 'p_obst':obst, 'obst_angle':angle, 'obst_speed':speed, 'subj_id':subj_id, 
                       'trial_id':trial_id, 'stimuli_onset':onset, 'stimuli_out': stimuli_out, 'time_stamp':time_stamp, 'ps_trial': ps_trial,
                         'ps_subj': 0})
            head = tail + 1
ps_subj = [0] * n_subj
for i in range(len(data.trajs)):
    if data.info['obst_speed'][i] == 0:
        subj_id = data.info['subj_id'][i]
        ps_subj[subj_id] += data.info['ps_trial'][i] / 10
for i in range(len(data.trajs)):
    subj_id = data.info['subj_id'][i]
    data.info['ps_subj'][i] = ps_subj[subj_id]
data.filter_all()
outfile = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data.pickle'))       
with open(outfile, 'wb') as f:   
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)  

In [5]:
len(data.trajs)

640